In [5]:
import pandas as pd
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
import re
from datetime import datetime
from selenium.webdriver.chrome.options import Options
import random
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime, timedelta
import locale



# Récupération de la data

In [20]:
locale.setlocale(locale.LC_TIME, 'fr_FR.UTF-8')

# Liste de User-Agent
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36 Edge/91.0.864.67",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Firefox/89.0",
    "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:51.0) Gecko/20100101 Firefox/51.0",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:91.0) Gecko/20100101 Firefox/91.0",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36",
    "Mozilla/5.0 (Linux; Android 9; SM-G960F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.115 Mobile Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36"
]

def get_date(date_start):
    date = datetime.now()
    date_start = datetime.strptime(date_start, "%Y-%m-%d %H:%M:%S")

    dates = []

    while date_start < date - timedelta(days=1):
        dates.append(date_start)
        date_start += timedelta(days=1)

    return dates
    
    

# Fonction pour initialiser le driver avec un User-Agent donné
def create_driver_with_user_agent(user_agent):
    chrome_options = Options()
    chrome_options.add_argument(f"user-agent={user_agent}")
    chrome_options.add_argument("--headless")  # Pour exécuter sans fenêtre
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--window-size=1920,1080")
    
    # Crée le driver avec les options
    driver = webdriver.Chrome(options=chrome_options)
    return driver

# Liste pour stocker les données
data = []

dates = get_date('2024-11-12 00:00:00')
print(dates)
# Pour chaque lien, essayez différents User-Agent en cas d'échec
for date in dates:

    jour = date.day
    mois = date.strftime('%B').replace('Ã©', 'e').replace('Ã»', 'u')
    annee = date.year

    print(jour, mois, annee)

    if jour == 1:
        jour = '1er'

    # user_agent = random.choice(USER_AGENTS)  # Choisir un User-Agent au hasard
    driver = create_driver_with_user_agent(USER_AGENTS[0])

    # Essayer plusieurs fois avec différents User-Agent si le tableau n'est pas trouvé
    retries = 3
    for attempt in range(retries):
        try:
            driver.get(f"https://www.infoclimat.fr/observations-meteo/archives/{jour}/{mois}/{annee}/rennes-st-jacques/07130.html",)
            
            WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "resptable-releves")))

            
            # Essayer de trouver le tableau
            tab = driver.find_element(By.ID, 'resptable-releves')
            thead = tab.find_element(By.TAG_NAME, 'thead').find_element(By.TAG_NAME, 'tr').find_elements(By.TAG_NAME, 'th')
            col = [i.text for i in thead]
            tbody = tab.find_element(By.TAG_NAME, 'tbody').find_elements(By.TAG_NAME, 'tr')

            # Ajouter les données à la liste
            for tr in tbody:
                tbody_heur = tr.find_element(By.TAG_NAME, 'th')
                tbody_date = tbody_heur.find_element(By.TAG_NAME, 'span').get_attribute('original-title')
                date_match = re.search(r'\d{2}/\d{2}/\d{4}', tbody_date)
                if date_match:
                    date_jour = date_match.group()                
                else:
                    date_jour = ""
                    
                try:
                    # Format : dd/mm/yyyy hh
                    date_time = datetime.strptime(f"{date_jour} {tbody_heur.text.replace('h', '')}", "%d/%m/%Y %H")
                except ValueError:
                    date_time = None  # Gérer les erreurs de conversion en datetime

                data_temp = [i.text for i in tr.find_elements(By.TAG_NAME, 'td')]
                data_temp.insert(0, date_time)
                data.append(data_temp)

            print(f"La page du {jour} {mois} {annee} a été scrap !")
            break  # Sortir de la boucle en cas de succès

        except Exception as e:
            if attempt == retries - 1:
                print("Échec après plusieurs tentatives, passer au lien suivant.")
            else:
                # Choisir un autre User-Agent pour la prochaine tentative
                user_agent = random.choice(USER_AGENTS)
                driver.quit()
                driver = create_driver_with_user_agent(user_agent)
df = pd.DataFrame(data, columns=col)
df.to_csv(f'../dataset/meteo_rennes_start_{dates[0].strftime("%d-%m-%Y")}.csv')


[datetime.datetime(2024, 11, 12, 0, 0)]
12 novembre 2024
La page du 12 novembre 2024 a été scrap !


In [2]:
# col = ['Heure locale\naccess_time\n30mn\nMETAR', '', 'Température', 'Temps', 'Pluie', 'Vent', 'Humidité', 'Bio-météo', 'Pt. de rosée', 'Pression', 'Visibilité']

df = pd.DataFrame(data, columns=col)


NameError: name 'pd' is not defined

,Heure locale\naccess_time\n30mn\nMETAR,,Température,Temps,Pluie,Vent,Humidité,Bio-météo,Pt. de rosée,Pression,Visibilité
0,2023-01-01 00:00:00,add_circle_outline,9.5 °C\n9.5 → 9.9,,0 mm/1h,7 km/h\nraf.10.8,94%,8.5,8.6 °C,1013.3hPa,60 km
1,2023-01-01 23:00:00,add_circle_outline,9.9 °C\n9.9 → 10.5,,0 mm/1h,7 km/h\nraf.28.8,94%,9,9 °C,1012.8hPa,60 km
2,2023-01-01 22:00:00,add_circle_outline,10.5 °C\n10.5 → 13.1,,0 mm/1h,22 km/h\n\nraf.42.5,91%,,9.1 °C,1011.6hPa,60 km
3,2023-01-01 21:00:00,add_circle_outline,13.1 °C\n13 → 13.4,,0 mm/1h,22 km/h\n\nraf.39.2,89%,,11.3 °C,1010.1hPa,14 km
4,2023-01-01 20:00:00,add_circle_outline,13.0 °C\n12.1 → 13,,0 mm/1h,18 km/h\n\nraf.34.9,89%,,11.2 °C,1008.6hPa,30 km


In [1]:
df.to_csv(f'dataset/meteo_rennes_start_{dates[0].strftime("%d-%m-%Y")}.csv')

NameError: name 'df' is not defined